# Início

In [ ]:
# ---- Importações ----
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
import os
import pickle

from datetime import datetime
from darts.models import RandomForestModel
from darts import TimeSeries

In [2]:
# ---- Configurações ----
codigo = 'BRL=X'
inicio = '2000-01-01'
fim = pd.to_datetime(datetime.now()).strftime('%Y-%m-%d')

data = yf.download(codigo, start=inicio, end=fim, multi_level_index=False)

/tmp/ipykernel_9070/2497561386.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(codigo, start=inicio, end=fim, multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


In [12]:
# ---- Carregando Dados ----
df = pd.DataFrame(data)

# Transformando dados
df = df[['Close']].reset_index()
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])
df['y'] = df['y'].astype(float)
df = df.dropna()
df = df.sort_values('ds').reset_index(drop=True)

series = TimeSeries.from_dataframe(df, 'ds', 'y', freq='D', fill_missing_dates=True)

In [13]:
# ---- Separando Dados ----
split = int(len(series) * 0.8)
train = series[:split]
test = series[split:]

In [15]:
# ---- Modelo ----
model = RandomForestModel(
    lags=14,
    n_estimators=100,
    random_state=42,
    criterion='absolute_error',
)

model.fit(train)
prediction = model.predict(len(test))

ValueError: Input y contains NaN.

### Salvando modelo

In [ ]:
os.makedirs('../Models/RandomForest', exist_ok=True)

In [ ]:
with open('../Models/RandomForest/random_forest_v1.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!


### Carregando modelo

In [ ]:
def load_neural_model():
    with open('../Models/RandomForest/random_forest_v1.pkl', 'rb') as f:
        loaded_model = pickle.load(f)
    
    return loaded_model

loaded_model = load_neural_model()